In [1]:
import pygame
import pygame_gui
from pygame_gui.windows.ui_file_dialog import UIFileDialog
from pygame_gui.elements.ui_button import UIButton
from pygame.rect import Rect
import tensorflow as tf
import numpy as np
import sklearn as sk
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import pathlib
import time
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys
import os
tf.get_logger().setLevel('ERROR') #verbose=0



pygame 2.1.3.dev8 (SDL 2.26.2, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.mixer.init(44100, -16, 2, 512) 
pygame.mixer.set_num_channels(32)
pygame.init()

pygame.mixer.music.load("assets/music.ogg") #load music
siren = pygame.mixer.Sound("assets/siren.wav") #load jump sound and assign it to "jump"
pygame.mixer.music.set_volume(0.5)
siren.set_volume(0.2)
black = (0,0,0) #Colors
white = (255,255,255)
red = (255,0,0)
sysfont = pygame.font.get_default_font()
font = pygame.font.SysFont(sysfont,100) 
model = load_model('model/gorillamodel.h5')


batch_size = 32
img_height = 180
img_width = 180
class_names = ['Gorilla','Human']

windres = (1200,1200*0.75)


In [3]:
def gorcheck(x):
    img = tf.keras.utils.load_img(x, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions)
    shimg = pygame.image.load(x).convert()
    shimg = pygame.transform.scale(shimg,(128,128))
    return score,shimg

In [4]:
def gamestart():
    flch=0
    window_surface = pygame.display.set_mode(windres)
    background = pygame.image.load('assets/bg.jpg').convert()
    background = pygame.transform.scale(background,windres)
    manager = pygame_gui.UIManager(windres)
    clock = pygame.time.Clock()
    pygame.display.set_caption("Gorilla Catcher") #sets the name of the tab
    
    file_selection_button = UIButton(relative_rect=Rect((windres[0]/2)-65, windres[1]/4, 130, 50),
                                    manager=manager, text='Select Image')

    img_height = 180
    img_width = 180
    bild = ''
    pygame.mixer.music.play(loops=-1)

    while 1:
        time_delta = clock.tick(60) / 1000.0

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                quit()
            if event.type == pygame.USEREVENT:
                if event.user_type == pygame_gui.UI_BUTTON_PRESSED:
                    if event.ui_element == file_selection_button:
                        file_selection = UIFileDialog(rect=Rect(0, 0, 400, 800), manager=manager, allow_picking_directories=True)
                                                               #0 #0 #300 #300
                    if event.ui_element == file_selection.ok_button:
                        bild = file_selection.current_file_path
                        classc,imgrend = gorcheck(bild)
                        flch = 0
            manager.process_events(event)
        manager.update(time_delta)
        window_surface.blit(background, (0, 0))

        if len(str(bild)) > 0:
            if class_names[np.argmax(classc)] == 'Gorilla':
                while flch == 0:
                    pygame.mixer.Sound.play(siren)
                    flch = 1
                classrender = font.render(class_names[np.argmax(classc)], True,black)
                text_rect = classrender.get_rect(center=(windres[0]/2, windres[1]/2))
                pygame.draw.rect(window_surface, red, pygame.Rect(text_rect))            
            if class_names[np.argmax(classc)] == 'Human':
                classrender = font.render(class_names[np.argmax(classc)], True,white)
                text_rect = classrender.get_rect(center=(windres[0]/2, windres[1]/2))
                pygame.draw.rect(window_surface, black, pygame.Rect(text_rect))            

            
            imgrend2 = pygame.image.load(bild).convert() # Load the image
            imgrend2 = pygame.transform.scale(imgrend2,(360,360)) # rescale it 
            imgrend3 = pygame.transform.scale(imgrend2,(380,380)) # create a different scale, which we use for the border
            imgret1 = imgrend2.get_rect(center=(windres[0]/2, windres[1]/1.3)) # where to blit image
            imgret2 = imgrend3.get_rect(center=(windres[0]/2, windres[1]/1.3)) # where to blit rectangle
            if class_names[np.argmax(classc)] == 'Gorilla':
                pygame.draw.rect(window_surface, red, pygame.Rect(imgret2))
            if class_names[np.argmax(classc)] == 'Human':
                pygame.draw.rect(window_surface, black, pygame.Rect(imgret2))
            window_surface.blit(classrender, text_rect) 
            window_surface.blit(imgrend2,imgret1) 
            
            
        manager.draw_ui(window_surface)
        pygame.display.update()

In [5]:
gamestart()

/var/folders/8_/bnynf45x5l10217ytky1647r0000gn/T/ipykernel_31364/2570401678.py:25: DeprecationWarning: Pygame GUI event types can now be used directly as event.type rather than event.user_type. This old style user_type event will go away in version 0.8.0
  if event.user_type == pygame_gui.UI_BUTTON_PRESSED:


1/1 [==============================] - 0s 101ms/step


2023-01-11 17:18:59.219225: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 14ms/step
